In [1]:
import tensorflow as tf
from tensorflow import keras
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from keras.applications import ResNet50V2

tf.__version__

'2.10.0'

In [2]:
PATH = 'data'

os.listdir(PATH)

['archive (12).zip',
 'jpeg224',
 'sample_submission.csv',
 'test.csv',
 'train.csv',
 'train_split.csv',
 'val_split.csv',
 'subset.csv']

In [3]:
train = pd.read_csv(f'{PATH}/subset.csv')

val = pd.read_csv(f'{PATH}/val_split.csv')

test = pd.read_csv(f'{PATH}/test.csv')

train.shape, val.shape, test.shape

((2220, 8), (10932, 8), (10982, 5))

In [4]:
train.head()

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target
0,ISIC_0533349,IP_5208504,female,45.0,lower extremity,unknown,benign,0
1,ISIC_8814612,IP_0414408,male,50.0,torso,unknown,benign,0
2,ISIC_6515241,IP_6245507,male,45.0,lower extremity,unknown,benign,0
3,ISIC_5075261,IP_2117218,male,40.0,upper extremity,unknown,benign,0
4,ISIC_2624460,IP_1969685,male,50.0,torso,unknown,benign,0


In [8]:
BATCH_SIZE = 64

AUTO = tf.data.experimental.AUTOTUNE

def decode(name, label):
    img = tf.io.read_file(name)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.cast(img, tf.float32) 
    return img, label


def load_ds(df):
    options = tf.data.Options()
    options.experimental_deterministic = False
    imgs, labels = df["image_name"].values, df["target"].values
    imgs = [f'{PATH}/jpeg224//train/{name}.jpg' for name in imgs]
    ds = tf.data.Dataset.from_tensor_slices((imgs, labels))
    ds = ds.with_options(options)
    ds = ds.map(decode, num_parallel_calls=AUTO)
    ds = ds.cache()
    ds = ds.shuffle(2048)
    ds = ds.batch(BATCH_SIZE)
    #ds = ds.prefetch(buffer_size=AUTO)
    return ds

In [9]:
train_ds = load_ds(train)
val_ds = load_ds(val)

In [ ]:
for img, label in train_ds.take(1):
    print(img)
    print(label)

In [11]:
IMAGE_SIZE = (224, 224, 3)

encoder = ResNet50V2(
    include_top=False,
    input_shape=IMAGE_SIZE,
    weights='imagenet'
)

encoder.trainable = False

inputs = tf.keras.Input(shape=IMAGE_SIZE)
x = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)(inputs)
x = encoder(x, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.Model(inputs, outputs)
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 rescaling (Rescaling)       (None, 224, 224, 3)       0         
                                                                 
 resnet50v2 (Functional)     (None, 7, 7, 2048)        23564800  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 1)                 2049      
                                                                 
Total params: 23,566,849
Trainable params: 2,049
Non-trainable params: 23,564,800
_____________________________________________

In [12]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=[keras.metrics.AUC(name='auc')]
)

In [14]:
filepath = './checkpoints/checkpoints'

cb = tf.keras.callbacks.ModelCheckpoint(
    filepath = filepath,
    monitor = "val_auc",
    verbose = 0,
    save_best_only = True,
    save_weights_only = True,
    mode = "max"
)

In [15]:
model.fit(
    train_ds, 
    epochs=10, 
    validation_data=val_ds, 
    validation_steps=10,
    callbacks = [cb])

Epoch 1/10
35/35 [==============================] - 122s 4s/step - loss: 0.0586 - auc: 0.9032 - val_loss: 0.1038 - val_auc: 0.7073
Epoch 2/10
35/35 [==============================] - 123s 4s/step - loss: 0.0566 - auc: 0.9160 - val_loss: 0.0898 - val_auc: 0.8035
Epoch 3/10
35/35 [==============================] - 121s 3s/step - loss: 0.0546 - auc: 0.9219 - val_loss: 0.0840 - val_auc: 0.7876
Epoch 4/10
35/35 [==============================] - 115s 3s/step - loss: 0.0526 - auc: 0.9304 - val_loss: 0.0798 - val_auc: 0.7515
Epoch 5/10
35/35 [==============================] - 119s 3s/step - loss: 0.0511 - auc: 0.9456 - val_loss: 0.0564 - val_auc: 0.8948
Epoch 6/10
35/35 [==============================] - 114s 3s/step - loss: 0.0504 - auc: 0.9404 - val_loss: 0.0702 - val_auc: 0.7253
Epoch 7/10
35/35 [==============================] - 116s 3s/step - loss: 0.0483 - auc: 0.9519 - val_loss: 0.0874 - val_auc: 0.6903
Epoch 8/10
35/35 [==============================] - 138s 4s/step - loss: 0.0463 - a

In [16]:
model.load_weights(filepath)
model.save('model.h5')

In [17]:
BATCH_SIZE = 256

def decode_test(name):
    img = tf.io.read_file(name)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.cast(img, tf.float32) 
    return img


def load_test_ds(df):
    imgs = df["image_name"].values
    imgs = [f'{PATH}/jpeg224/test/{name}.jpg' for name in imgs]
    ds = tf.data.Dataset.from_tensor_slices(imgs)
    ds = ds.map(decode_test)
    ds = ds.batch(BATCH_SIZE)
    return ds

In [18]:
test_ds = load_test_ds(test)

In [19]:
preds = model.predict(test_ds)

43/43 [==============================] - 404s 9s/step


In [20]:
preds

array([[0.00289533],
       [0.00232835],
       [0.00476324],
       ...,
       [0.01443924],
       [0.00195344],
       [0.00844666]], dtype=float32)

In [21]:
submission = pd.DataFrame({
    'image_name': test['image_name'].values,
    'target': preds.ravel()
})

submission.head()

,image_name,target
0,ISIC_0052060,0.002895
1,ISIC_0052349,0.002328
2,ISIC_0058510,0.004763
3,ISIC_0073313,0.000790
4,ISIC_0073502,0.001589


In [22]:
submission.to_csv('submission.csv', index=False)